In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import Delaunay
from scipy.sparse import csr_matrix

import mygene

from pathlib import Path

import matplotlib.pyplot as plt

In [2]:
sparse_LIBD_dataframe = pd.read_csv('../../spatialLIBD/LIBD_sparse_data.csv', header=0, index_col=0, names=['gene', 'spot', 'count'])

In [3]:
LIBD_gene_names = pd.read_csv('../../spatialLIBD/LIBD_gene_names.csv', header=0, index_col=0)

In [4]:
gene_names = LIBD_gene_names.values.flatten()
mg = mygene.MyGeneInfo()
gene_symbols = mg.querymany(gene_names, scopes='ensembl.gene', fields="symbol", species="human", as_dataframe=True)
gene_symbols = gene_symbols[~gene_symbols.index.duplicated(keep='first')]
unmatched_mask = pd.isna(gene_symbols)["symbol"].values
processed_gene_symbols = gene_symbols["symbol"].values.astype(str)
processed_gene_symbols[unmatched_mask] = gene_symbols.index[unmatched_mask]

querying 1-1000...done.
querying 1001-2000...done.
querying 2001-3000...done.
querying 3001-4000...done.
querying 4001-5000...done.
querying 5001-6000...done.
querying 6001-7000...done.
querying 7001-8000...done.
querying 8001-9000...done.
querying 9001-10000...done.
querying 10001-11000...done.
querying 11001-12000...done.
querying 12001-13000...done.
querying 13001-14000...done.
querying 14001-15000...done.
querying 15001-16000...done.
querying 16001-17000...done.
querying 17001-18000...done.
querying 18001-19000...done.
querying 19001-20000...done.
querying 20001-21000...done.
querying 21001-22000...done.
querying 22001-23000...done.
querying 23001-24000...done.
querying 24001-25000...done.
querying 25001-26000...done.
querying 26001-27000...done.
querying 27001-28000...done.
querying 28001-29000...done.
querying 29001-30000...done.
querying 30001-31000...done.
querying 31001-32000...done.
querying 32001-33000...done.
querying 33001-33538...done.
Finished.
2 input query terms found 

In [5]:
def create_subset_matrix(dataframe, sample_id, coordinates, labels, sample_name):
    subset_mask = (sample_id["x"] == sample_name)
    subset_index = sample_id[subset_mask].index
    subset = dataframe[dataframe["spot"].isin(subset_index)]

    subset_coordinates = coordinates[subset_mask.values]
    subset_labels = labels[subset_mask.values]
    
    rows, columns = len(subset["spot"].unique()), dataframe["gene"].max()
    
    LIBD_data = csr_matrix((subset["count"], (subset["spot"] - subset["spot"].min(), subset["gene"] -1)), shape=(rows, columns)).toarray()
    
    return LIBD_data, subset_coordinates, subset_labels

In [6]:
LIBD_sample_ids = pd.read_csv('../../spatialLIBD/LIBD_sample_id.csv', header=0, index_col=0)
LIBD_spatial_coordinates = pd.read_csv('../../spatialLIBD/LIBD_coordinates.csv', header=0)
LIBD_labels = pd.read_csv('../../spatialLIBD/LIBD_annotations.csv', header=0)

sample_names = LIBD_sample_ids["x"].unique()
num_samples = len(sample_names)

data_directory = Path("../../data/spatialLIBD/files")
data_directory.mkdir(parents=True, exist_ok=True)

In [7]:
def get_neighborhood(spot_locations, method="delaunay", max_neighbors=5, percentile_threshold=95):
    """Return adjacency matrix in sparse COO format (esssentially).
    """

    num_spots = len(spot_locations)
    adjacency_matrix = None
    np.random.seed(0)
    if method == "nearest":
        edges=[]
        rng = np.random.default_rng()
        for vertex in range(num_spots):
            neighbors = rng.choice(num_spots, size=num_spots // 2, replace=False)
            distances = np.linalg.norm(spot_locations[vertex] - spot_locations[neighbors], axis=1)
            top_neighbor_indices = np.argsort(distances)[:max_neighbors]
            top_neighbors = neighbors[top_neighbor_indices]
            edges.extend([(vertex, neighbor) for neighbor in top_neighbors if vertex != neighbor]) 
    elif method == "delaunay":
        triangulation = Delaunay(spot_locations)
        indptr, indices = triangulation.vertex_neighbor_vertices
        edges = []
        for vertex in range(num_spots):
            neighbors = indices[indptr[indices[vertex] : indices[vertex + 1]]]
            distances = np.linalg.norm(spot_locations[vertex] - spot_locations[neighbors], axis=1)
            top_neighbor_indices = np.argsort(distances)[:max_neighbors]   
            top_neighbors = neighbors[top_neighbor_indices]
            edges.extend([(vertex, neighbor) for neighbor in top_neighbors if vertex != neighbor])

    # Trim the top 10% of edges.
    distances = np.array([np.linalg.norm(spot_locations[source] - spot_locations[destination]) for (source, destination) in edges])
    threshold_mask = (distances < np.percentile(distances, percentile_threshold))
    adjacency_matrix = np.array(edges)[threshold_mask]

    return adjacency_matrix

In [8]:
def write_spicemix_input(spot_data, gene_names, coordinates, labels, sample_name, output_directory):
    """Write input files in correct format for SpiceMix.

    """
    
    output_directory.mkdir(parents=True, exist_ok=True)
    expression_filename = "expression_{}.txt".format(sample_name)
    neighborhood_filename = "neighborhood_{}.txt".format(sample_name)
    coordinates_filename = "coordinates_{}.txt".format(sample_name)
    gene_names_filename = "genes_{}.txt".format(sample_name)
    labels_filename = "labels_{}.txt".format(sample_name)

    pd.DataFrame(coordinates).to_csv(Path(output_directory) / coordinates_filename, sep="\t", header=False, index=False)
    pd.DataFrame(labels).to_csv(Path(output_directory) / labels_filename, sep="\t", header=False, index=False)
    
    adjacency_matrix = get_neighborhood(coordinates, method="nearest")
    pd.DataFrame(adjacency_matrix).to_csv(Path(output_directory) / neighborhood_filename, sep="\t", header=False, index=False)

    total_spot_data = spot_data.sum(axis=1)
    transformed_gene_expression_data = np.log(1 + 10**4 * (spot_data / total_spot_data[:, np.newaxis]))
    
    pd.DataFrame(transformed_gene_expression_data).to_csv(Path(output_directory) / expression_filename, sep="\t", header=False, index=False)

    with open(Path(output_directory) / gene_names_filename, 'w') as filehandle:
        for gene in gene_names:
            filehandle.write('%s\n' % gene)
            
    return adjacency_matrix

In [9]:
spot_data = []
spot_coordinates = []
spot_labels = []
for sample_name in sample_names:
    sample_spot_data, subset_coordinates, subset_labels = create_subset_matrix(sparse_LIBD_dataframe, LIBD_sample_ids, LIBD_spatial_coordinates, LIBD_labels, sample_name)
    spot_data.append(sample_spot_data)
    spot_coordinates.append(subset_coordinates)
    spot_labels.append(subset_labels)

In [10]:
# Filtering genes with less than 20% non-zeros across all samples
threshold = 0.1
nonzero_mask = np.full(sparse_LIBD_dataframe["gene"].max(), True)
for sample_spot_data in spot_data:
    sample_nonzero_mask = ((sample_spot_data != 0).sum(axis=0) / len(sample_spot_data) > threshold)
    nonzero_mask = (nonzero_mask & sample_nonzero_mask)


print("Number of HVGs: %d" % nonzero_mask.sum())
filtered_spot_data = [sample_spot_data[:, nonzero_mask] for sample_spot_data in spot_data]
filtered_genes = processed_gene_symbols[nonzero_mask]

Number of HVGs: 3194


In [11]:
len(filtered_genes)
len(np.unique(filtered_genes))

3194

In [12]:
print(filtered_genes)

['HES4' 'ISG15' 'SDF4' ... 'ND6' 'CYTB' 'LOC102724770']


In [ ]:
width = 3
height = num_samples // width + (num_samples % width != 0)
# width = 1
# height = 1
fig, axes= plt.subplots(height, width, figsize=(height * 10, width * 10), squeeze=False)
plt.gca().set_aspect('equal', adjustable='box')

for coordinates, labels, sample_spot_data, sample_name, ax in zip(spot_coordinates, spot_labels, filtered_spot_data, sample_names, axes.flat):
    spot_locations = coordinates[["array_row", "array_col"]].values
    labels = labels["x"].values.astype(str)

    adjacency_matrix = write_spicemix_input(sample_spot_data, filtered_genes, spot_locations, labels, sample_name, data_directory)
    
    for (source, destination) in adjacency_matrix:
        ax.plot([spot_locations[source, 0], spot_locations[destination, 0]],
            [spot_locations[source, 1], spot_locations[destination, 1]], color="gray", linewidth=1)

    top_ax = ax.twinx()
    top_ax.set_zorder(2)
    ax.set_zorder(1)
        
    _, integer_labels = np.unique(labels, return_inverse=True)
    x, y = spot_locations.T
    ax.scatter(x, y, s=3, c=integer_labels)
    ax.set_xlim(ax.get_xlim())
    top_ax.set_ylim(ax.get_ylim())

Number of HVGs: 3194
